<h1 align='center'> Simulation of a plasma wakefield accelerator (PWFA)</h1>
<center>
Stephen D. Webb and David Bruhwiler <br>
RadiaSoft LLC <br>
swebb@radiasoft.net and bruhwiler@radiasoft.net</center>

Developed for a project supported by the United States Department of Energy, Office of Science, Office of High Energy Physics under contract number DE-SC0018718.

***
## Introduction

This notebook models a beam-driven plasma wakefield accelerator (PWFA) using nominal FACET-II parameters.

Particle groups:
 - electron plasma
 - protons (optional)
 - electron drive beam
 - witness beam
***

In [2]:
from __future__ import print_function
import shutil, os, h5py

import numpy as np
from scipy import constants
from scipy.special import erfc, k0, k1

# Imports specific to the FBPIC code
from fbpic.main import Simulation
from fbpic.openpmd_diag import FieldDiagnostic, ParticleDiagnostic, \
     ParticleChargeDensityDiagnostic,\
     set_periodic_checkpoint, restart_from_checkpoint
from fbpic.lpa_utils.bunch import add_elec_bunch_gaussian
from fbpic.fields.smoothing import BinomialSmoother

***
## Simulation Parameters

The simulation uses a moving window, beginning with the drive bunch outside the plasma.

The plasma has a density $n_e$, with the local plasma frequency given by $\omega_p = \sqrt{\frac{4 \pi n_e e^2}{m_e}}$ for the electron charge $e$ and mass $m_e$. The plasma wavenumber is $k_p = \omega_p / c$.

In [3]:
## Domain physical parameters
print("Some physical parameters...\n")

## Plasma channel parameters
n_plasma = 4.e16  # cm^-3

# convert to per cubic meter
n_plasma *= 100**3
print("n_plasma = ", n_plasma, " [m^-3]")

# derived plasma quantities
omega_p = np.sqrt(n_plasma*constants.elementary_charge**2/(constants.m_e*constants.epsilon_0))
k_p = omega_p/constants.c

lambda_p = 2.*np.pi/k_p
print("lambda_p = ", lambda_p*1.e6, " [microns]")

## Beam parameters
# Drive bunch is gaussian
# drive_sigma_r = 3.65e-6  # meters
drive_sigma_r = 0.4 / k_p  # meters
print("R_drive_rms = ", drive_sigma_r*1.e6, " [microns]")

# drive_sigma_z = 12.77e-6  # meters
drive_sigma_z = 1.2 / k_p
print("Z_drive_rms = ", drive_sigma_z*1.e6, " [microns]")

# drive_Q = 1.e10*(-1.*constants.elementary_charge)   # Coulombs
drive_Q = 3.e-9   # Coulombs
print("drive_Q = ", drive_Q*1.e9, " [nC]")

# Calculate peak beam density
drive_volume_rms = 2.*np.pi*drive_sigma_z*drive_sigma_r**2
drive_dens_rms = drive_Q/np.abs(constants.elementary_charge)/drive_volume_rms
print("n_beam_peak_rms = ", drive_dens_rms, " [m^-3]")
print("n_beam/n_plasma = ", drive_dens_rms/n_plasma)

drive_N_macro = 16000
drive_gamma = 1.957e+4

# Witness bunch, also gaussian
witness_sigma_r = 2.e-6 #meters
witness_sigma_z = 6.e-6  # meters
witness_Q = 1.e-10*(-1.*constants.elementary_charge)   # Coulombs
witness_N_macro = 1600
witness_gamma = 100

trailing_distance = 150.e-6 # meters

## Domain parameters

# Domain size, include the whole thing and some trailing distance
domain_length = 2.*lambda_p  # meters
domain_radius = lambda_p  # meters

# Grid size, resolve the drive bunch
Delta_r = min(0.244*drive_sigma_r, 0.2*lambda_p)  # meters
Delta_z = min(Delta_r, min(0.05*drive_sigma_z, 0.1*lambda_p))  # meters
print(" ")
print("Delta_z = ", Delta_z*1e6, "[microns]")
print("Delta_r = ", Delta_r*1e6, "[microns]")

# Derived quantities
Nz = int(np.rint(domain_length/Delta_z))
Nr = int(np.rint(domain_radius/Delta_r))
print("Nz = ", Nz)
print("Nr = ", Nr)

dt = (np.sqrt((Delta_z**2 + Delta_r**2))/constants.c)  # sec

# start the ramp after the drive bunch has existed a while
ramp_start = domain_length
ramp_length = 5.*drive_sigma_z

# We want to run the simulation just long enough for the fields to form behind the drive bunch
sim_time = 2.5*domain_length/constants.c

dump_period = ( int(sim_time/dt) + 8 ) / 8
Nsteps = 8*dump_period+1
print("Nsteps = ", Nsteps)

# Simplest case -- cylindrical symmetry
Nm = 1

Some physical parameters...

n_plasma =  4e+22  [m^-3]
lambda_p =  166.9471636350277  [microns]
R_drive_rms =  10.62818653107447  [microns]
Z_drive_rms =  31.884559593223404  [microns]
drive_Q =  3.0  [nC]
n_beam_peak_rms =  8.274318570057662e+23  [m^-3]
n_beam/n_plasma =  20.685796425144154
 
Delta_z =  1.5942279796611702 [microns]
Delta_r =  2.5932775135821706 [microns]
Nz =  209
Nr =  64
Nsteps =  281


***
## The Simulation

The FBPIC simulation is started in the cell below.
***

In [4]:
# remove old data, if necessary
if os.path.exists('./diags/hdf5'):
    shutil.rmtree('./diags/hdf5')

# Create a field smoother
n_passes_z=1
n_passes_r=1
dict_pass = {"z": n_passes_z, "r": n_passes_r}
comp_z = False
comp_r = False
dict_comp = {"z": comp_z, "r": comp_r}
my_smoother=BinomialSmoother(n_passes=dict_pass, compensator=dict_comp)

# Create the simulation
my_sim = Simulation(Nz, domain_length, Nr, domain_radius, Nm, dt, 
                    boundaries='open', particle_shape='linear', verbose_level=2,
                    n_order=-1, smoother=my_smoother, current_correction="curl-free")

# By default the simulation initializes an electron species (my_sim.ptcl[0])
# Because we did not pass the arguments `n`, `p_nz`, `p_nr`, `p_nz`,
# this electron species does not contain any macroparticles.
# It is okay to just remove it from the list of species.
my_sim.ptcl = []

# create the uniform density function for the plasma electrons
def dens_func_elec(z, r) :
    """Returns relative density at position z and r"""
    # Allocate relative density
    n = np.ones_like(z)
    # Make linear ramp
    n = np.where( z < ramp_start + ramp_length, (z-ramp_start)/ramp_length, n )
    # Supress density before the ramp
    n = np.where( z < domain_length + ramp_start, 0., n )
    return(n)

# plasma electrons
e_plasma = my_sim.add_new_species(q = -1.*constants.elementary_charge,
                                  m = constants.electron_mass,
                                  dens_func = dens_func_elec, 
                                  n = n_plasma, p_nz=4, p_nr=4, p_nt = 4*Nm)

# add the Gaussian drive beam
add_elec_bunch_gaussian(my_sim, 
                        sig_r = drive_sigma_r, 
                        sig_z = drive_sigma_z, 
                        n_emit=0., 
                        gamma0=drive_gamma, 
                        sig_gamma=1.,
                        Q=drive_Q, 
                        N=drive_N_macro, 
                        tf=0.0, 
                        zf=.75*domain_length, boost=None)

# add the witness bunch
add_elec_bunch_gaussian(my_sim, 
                        sig_r = witness_sigma_r, 
                        sig_z = witness_sigma_z, 
                        n_emit=0., 
                        gamma0=drive_gamma, 
                        sig_gamma=1.,
                        Q=witness_Q, 
                        N=witness_N_macro, 
                        tf=0.0, 
                        zf=.75*domain_length-trailing_distance, boost=None)

# create the uniform density function for the plasma ions
def dens_func_ions(z, r) :
    """Returns relative density at position z and r"""
    # Allocate relative density
    n = np.ones_like(z)
    # Make linear ramp
    n = np.where( z < ramp_start + ramp_length, (z-ramp_start)/ramp_length, n )
    # Supress density before the ramp
    n = np.where( z < domain_length + ramp_start, 0., n )
    # Confine ions to be near the axis (the only ones that will move)
    n = np.where( r < 2.*drive_sigma_r, n, 0. )
    return(n)

# plasma ions
use_protons = False
if use_protons:
    protons = my_sim.add_new_species(q = constants.elementary_charge,
                                     m = constants.proton_mass,
                                     dens_func = dens_func_elec, 
                                     n = n_plasma, p_nz=4, p_nr=4, 
                                     p_nt = 4*Nm)

# Set the moving window
my_sim.set_moving_window(v = constants.c)

# Add diagnostics
if use_protons:
    my_sim.diags = [FieldDiagnostic(dump_period, my_sim.fld, comm=my_sim.comm),
                    ParticleDiagnostic(dump_period,
                            {"plasma": e_plasma, "beam": my_sim.ptcl[1], 
                             "witness": my_sim.ptcl[2], "protons": protons},
                             comm=my_sim.comm),
                    ParticleChargeDensityDiagnostic(dump_period, my_sim,
                            {"plasma": e_plasma, "beam": my_sim.ptcl[1], 
                             "witness": my_sim.ptcl[2], "protons": protons})
    ]
else:
    my_sim.diags = [FieldDiagnostic(dump_period, my_sim.fld, comm=my_sim.comm),
                    ParticleDiagnostic(dump_period,
                            {"plasma": e_plasma, "beam": my_sim.ptcl[1], 
                             "witness": my_sim.ptcl[2]},
                             comm=my_sim.comm),
                    ParticleChargeDensityDiagnostic(dump_period, my_sim,
                            {"plasma": e_plasma, "beam": my_sim.ptcl[1], 
                             "witness": my_sim.ptcl[2]})
    ]

# run the simulation
my_sim.step(Nsteps)


FBPIC (0.10.1)

MPI available: Yes
MPI processes used: 1
MPI Library Information: 
Open MPI v2.1.1, package: Open MPI mockbuild@buildhw-10.phx2.fedoraproject.org Distribution, ident: 2.1.1, repo rev: v2.1.0-100-ga2fdb5b, May 10, 2017 
CUDA available: No
Compute architecture: CPU
CPU multi-threading enabled: Yes
Threads: 40
FFT library: pyFFTW

PSATD stencil order: infinite
Particle shape: linear
Longitudinal boundaries: open
Transverse boundaries: reflective
Guard region size: 64 cells
Damping region size: 64 cells
Injection region size: 32 cells
Particle exchange period: every 7 step
Boosted frame: False

Calculating initial space charge field...
Done.

Calculating initial space charge field...
Done.

|███████████████████████████████████| 281/281, 0:00:00 left, 619 ms/step
Total time taken (with compilation): 0:03:12
Average time per iteration (with compilation): 685 ms



In [5]:
# read the data
file = h5py.File('./diags/hdf5/data00000280.h5','r')
data = file.get('data/')
step = data.get('280')
ptcls = step.get('particles')
electrons = ptcls.get('plasma')
pos = electrons.get('position')

fields = step.get('fields')
rho = fields.get('rho')

# convert to number density
therho = rho[0,:,:]/constants.elementary_charge
# convert to cm^-3
therho /= 100.**3

Es = fields.get('E')
Ez = Es.get('z')
Er = Es.get('r')
theEz = Ez[0,:,:]
theEr = Er[0,:,:]

Bs = fields.get('B')
Bt = Bs.get('t')
theBt = Bt[0,:,:]

x = pos.get('x')
y = pos.get('y')
z = pos.get('z')

xPos = x[:]
yPos = y[:]
z = z[:]
r = np.sqrt(xPos**2 + yPos**2)

file.close()

***
Plotting
***

In [6]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.colors as colors

class MidpointNormalize(colors.Normalize):
    """
    Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)

    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

In [7]:
Rads = np.linspace(0., domain_radius, Nr)
zeta = np.linspace(0., domain_length, Nz)

# move zeta so zero is centered on the drive bunch
z_avg = np.average(z)
z_avg -= constants.c * dt*(Nsteps-1)

zeta -= z_avg

zz, RR = np.meshgrid(zeta, Rads)

fig = plt.figure()

plt.imshow(therho,extent=[k_p*zeta[0],k_p*zeta[-1], k_p*Rads[0], k_p*Rads[-1]], cmap='viridis', origin='lower')
plt.xlabel(r'$k_p \zeta$')
plt.ylabel(r'$r \quad [\mu m]$')
cbar = plt.colorbar(orientation='horizontal')
cbar.set_label(r'$n_e \quad [cm^{-3}]$')

plt.tight_layout()

plt.savefig('rho.png')


FigureCanvasNbAgg()

In [8]:
fig = plt.figure()

# Fix the midpoint to zero field

ezmax = np.amax(theEz*1.e-9)
ezmin = np.amin(theEz*1.e-9)
ezavg = 0.

plt.imshow(theEz*1.e-9,extent=[k_p*zeta[0], k_p*zeta[-1], k_p*Rads[0], k_p*Rads[-1]], 
           cmap='RdBu', origin='lower', norm=MidpointNormalize(midpoint=ezavg,vmin=ezmin, vmax=ezmax))
plt.xlabel(r'$k_p \zeta$')
plt.ylabel(r'$k_p r$')
cbar = plt.colorbar(orientation='horizontal')
cbar.set_label(r'$E_z \quad [GV/m]$')

plt.savefig('Ez.png')


FigureCanvasNbAgg()

In [9]:
fig = plt.figure()

Ez_lineout = theEz[1,:]*1.e-9

plt.plot(k_p* zeta, Ez_lineout)
plt.xlabel(r'$k_p \zeta$')
plt.ylabel(r'$E_z$ [GV/m]')


FigureCanvasNbAgg()

Text(0,0.5,'$E_z$ [GV/m]')

In [10]:
fig = plt.figure()

Fr = (theEr - constants.speed_of_light*(theBt))*1.e-9

frmax = np.amax(Fr)
frmin = np.amin(Fr)
fravg = 0.

plt.imshow(Fr,extent=[k_p*zeta[0], k_p*zeta[-1], k_p*Rads[0], k_p*Rads[-1]], 
           cmap='RdBu', origin='lower', norm=MidpointNormalize(midpoint=fravg,vmin=frmin, vmax=frmax))
plt.xlabel(r'$\zeta = z - c t \quad [\mu m]$')
plt.ylabel(r'$r \quad [\mu m]$')
cbar = plt.colorbar(orientation='horizontal')
cbar.set_label(r'$E_r - c B_\theta \quad [GV/m]$')
plt.title(r'Transverse force $E_r - c B_\theta$')
plt.tight_layout()

plt.savefig('Fr.png')

FigureCanvasNbAgg()

***
## References

> 1. C. B. Schroeder, D. H. Whittum, and J. S. Wurtele, "Multimode Analysis of the Hollow Plasma Channel Wakefield Accelerator", _Phys. Rev. Lett._ __82__, 1177 (1999). [https://doi.org/10.1103/PhysRevLett.82.1177](https://doi.org/10.1103/PhysRevLett.82.1177)

> 2. R. Lehe, M. Kirchen, I. A. Andriyash, B. B. Godfrey, and J.-L. Vay, "A spectral, quasi-cylindrical and dispersion-free Particle-In-Cell algorithm", _Comp. Phys. Comm._ __203__, pp. 66-82 (2016). [https://doi.org/10.1016/j.cpc.2016.02.007](https://doi.org/10.1016/j.cpc.2016.02.007)

> 3. C. Joshi _et al._ "Plasma wakefield acceleration experiments at FACET II", _Plasma Phys. Control. Fusion_ __60__, 3 (2018).

> 4. A. W. Chao, "Physics of Collective Beam Instabilities in High Energy Accelerators", John Wiley & Sons (1993)

> 5. C. A. Lindstrom _et al._ "Measurement of Transverse Wakefields Induced by a Misaligned Positron Bunch in a Hollow Channel Plasma Accelerator", _Phys. Rev. Lett._ __120__, 124802 (2018).